In [30]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
from hmmlearn import hmm
import string

In [31]:
# Character mapping for encryption/decryption
char_map = {char: i for i, char in enumerate(string.ascii_lowercase + " ")}
reverse_char_map = {i: char for char, i in char_map.items()}


In [32]:
# HMM configuration
n_components = 5  # Number of hidden states
n_iter = 100  # Number of iterations for model fitting

In [33]:
# Train HMM on dummy sequences (just for demonstration)
def train_hmm():
    model = hmm.MultinomialHMM(n_components=n_components, n_iter=n_iter)
    training_data = generate_training_data()
    model.fit(training_data)
    return model

In [34]:
# Generate dummy training data (random sequences)
def generate_training_data():
    sequences = []
    for _ in range(1000):
        sequence = [np.random.randint(0, len(char_map)) for _ in range(50)]
        sequences.append(sequence)
    return np.array(sequences).reshape(-1, 1)


In [35]:
# Encrypt text using the fitted HMM
def encrypt_text(model, text):
    text_obs = np.array([char_map[char] for char in text.lower() if char in char_map]).reshape(-1, 1)
    hidden_states = model.predict(text_obs)
    encrypted_text = ''.join([reverse_char_map[state] for state in hidden_states])
    return encrypted_text


In [36]:
# Decrypt the encrypted text
def decrypt_text(model, encrypted_text):
    encrypted_obs = np.array([char_map[char] for char in encrypted_text.lower() if char in char_map]).reshape(-1, 1)
    _, decrypted_states = model.decode(encrypted_obs)
    decrypted_text = ''.join([reverse_char_map[state] for state in decrypted_states])
    return decrypted_text

In [37]:
# Save text to file
def save_to_file(original_text, encrypted_text, decrypted_text, filename="encrypted_decrypted_text.txt"):
    with open(filename, 'w') as file:
        file.write(f"Original Text: {original_text}\n")
        file.write(f"Encrypted Text: {encrypted_text}\n")
        file.write(f"Decrypted Text: {decrypted_text}\n")


In [38]:
# Function to handle the encryption and decryption process
def process_text():
    original_text = text_input.get("1.0", "end-1c")
    if len(original_text) > 100:
        messagebox.showerror("Error", "Text exceeds 100 characters limit.")
        return
    
    if not original_text.strip():
        messagebox.showerror("Error", "Please enter some text.")
        return

  # Encrypt and decrypt the text
    encrypted_text = encrypt_text(model, original_text)
    decrypted_text = decrypt_text(model, encrypted_text)
    
    # Save to file
    save_to_file(original_text, encrypted_text, decrypted_text)
    
    # Show results in the GUI
    encrypted_output.delete("1.0", tk.END)
    decrypted_output.delete("1.0", tk.END)
    encrypted_output.insert(tk.END, encrypted_text)
    decrypted_output.insert(tk.END, decrypted_text)
    
    messagebox.showinfo("Success", "Text processed and saved to file.")

In [39]:
# Train the HMM model
model = train_hmm()


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [40]:
# Build GUI using Tkinter
root = tk.Tk()
root.title("Text Encryption & Decryption using HMM")


''

In [41]:
# Input frame
input_frame = tk.Frame(root)
input_frame.pack(pady=10)

tk.Label(input_frame, text="Enter text (max 100 characters):").pack(anchor="w")

text_input = tk.Text(input_frame, height=5, width=50)
text_input.pack()

In [42]:
# Output frame for Encrypted and Decrypted text
output_frame = tk.Frame(root)
output_frame.pack(pady=10)

tk.Label(output_frame, text="Encrypted Text:").pack(anchor="w")
encrypted_output = tk.Text(output_frame, height=3, width=50)
encrypted_output.pack()

tk.Label(output_frame, text="Decrypted Text:").pack(anchor="w")
decrypted_output = tk.Text(output_frame, height=3, width=50)
decrypted_output.pack()

# Button to trigger encryption and decryption
process_button = tk.Button(root, text="Encrypt & Decrypt", command=process_text)
process_button.pack(pady=10)

In [43]:
# Run the GUI loop
root.mainloop()